In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df=pd.read_csv('/content/ananadvihar(8hr).csv')

In [ ]:
data.head()

In [ ]:
print(data.head())

print(data.isnull().sum())

In [ ]:
sns.heatmap(data.isnull(), cbar=False, cmap='viridis')
plt.title("Missing Values in Dataset")
plt.show()

In [ ]:
def calculate_individual_aqi(concentration, breakpoints):

    for i in range(len(breakpoints) - 1):
        if concentration >= breakpoints[i][0] and concentration < breakpoints[i + 1][0]:
            C_low, C_high = breakpoints[i]
            I_low, I_high = breakpoints[i + 1]

            aqi = ((concentration - C_low) / (C_high - C_low)) * (I_high - I_low) + I_low
            return aqi

    return 500

def calculate_overall_aqi(row):

    pm25_breakpoints = [(0, 30), (30, 60), (60, 90), (90, 120), (120, 250), (250, 500)]
    pm10_breakpoints = [(0, 50), (50, 100), (100, 150), (150, 200), (200, 350), (350, 500)]
    so2_breakpoints = [(0, 40), (40, 80), (80, 380), (380, 800), (800, 1600), (1600, 3200)]
    no2_breakpoints = [(0, 40), (40, 80), (80, 180), (180, 280), (280, 400), (400, 600)]
    ozone_breakpoints = [(0, 50), (50, 100), (100, 140), (140, 180), (180, 240), (240, 300)]

    pm25_aqi = calculate_individual_aqi(row['PM2.5'], pm25_breakpoints)
    pm10_aqi = calculate_individual_aqi(row['PM10'], pm10_breakpoints)
    so2_aqi = calculate_individual_aqi(row['SO2'], so2_breakpoints)
    no2_aqi = calculate_individual_aqi(row['NO2'], no2_breakpoints)
    ozone_aqi = calculate_individual_aqi(row['Ozone'], ozone_breakpoints)


    overall_aqi = max(pm25_aqi, pm10_aqi, so2_aqi, no2_aqi, ozone_aqi)

    return overall_aqi


In [ ]:
imputer = SimpleImputer(strategy='median')
data_imputed = pd.DataFrame(imputer.fit_transform(data[['PM2.5', 'PM10', 'SO2', 'NO2', 'Ozone']]))
data_imputed.columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'Ozone']


In [ ]:
data_imputed['Overall_AQI'] = data_imputed.apply(calculate_overall_aqi, axis=1)


In [ ]:
data_imputed['lag_1'] = data_imputed['Overall_AQI'].shift(1)
data_imputed['lag_2'] = data_imputed['Overall_AQI'].shift(2)
data_imputed['lag_3'] = data_imputed['Overall_AQI'].shift(3)

In [ ]:
data_imputed = data_imputed.dropna()

In [ ]:
X = data_imputed[['lag_1', 'lag_2', 'lag_3']]
y = data_imputed['Overall_AQI']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [ ]:
xgb_model = xgb.XGBRegressor(n_estimators=1000,
                             learning_rate=0.01,
                             max_depth=6,
                             subsample=0.8,
                             colsample_bytree=0.8,
                             early_stopping_rounds=10,
                             objective='reg:squarederror')


In [ ]:
xgb_model.fit(X_train_scaled, y_train,
              eval_set=[(X_test_scaled, y_test)],
              verbose=True)

# Predictions
y_pred_xgb = xgb_model.predict(X_test_scaled)

# Evaluate model performance
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost Model - MSE: {mse_xgb:.2f}, R2: {r2_xgb:.2f}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test, label="True AQI", color='blue')
plt.plot(y_test.index, y_pred_xgb, label="XGBoost Prediction", color='red')
plt.legend()
plt.title('AQI Prediction Comparison using XGBoost')
plt.xlabel('Index')
plt.ylabel('AQI')
plt.show()

In [ ]:
param_grid = {
    'n_estimators': [100, 200, 500, 1000],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 6, 8, 10],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'alpha': [0, 0.1, 0.5, 1.0],
    'lambda': [0, 0.1, 0.5, 1.0]
}
xgb_model = xgb.XGBRegressor(objective='reg:squarederror')

grid_search = GridSearchCV(estimator=xgb_model,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           cv=3,
                           verbose=1,
                           n_jobs=-1)

grid_search.fit(X_train_scaled, y_train)
print("Best parameters found by GridSearchCV:", grid_search.best_params_)

best_xgb_model = grid_search.best_estimator_
y_pred_best_xgb = best_xgb_model.predict(X_test_scaled)

mse_best_xgb = mean_squared_error(y_test, y_pred_best_xgb)
r2_best_xgb = r2_score(y_test, y_pred_best_xgb)

print(f"Best XGBoost Model - MSE: {mse_best_xgb:.2f}, R2: {r2_best_xgb:.2f}")


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test, label="True AQI", color='blue')
plt.plot(y_test.index, y_pred_best_xgb, label="Best XGBoost Prediction", color='green')
plt.legend()
plt.title('AQI Prediction Comparison using Best XGBoost Model')
plt.xlabel('Index')
plt.ylabel('AQI')
plt.show()

In [ ]:
# Feature Importance
plt.figure(figsize=(10, 6))
xgb.plot_importance(best_xgb_model, max_num_features=10, importance_type='weight')
plt.title('Feature Importance')
plt.show()


In [ ]:
# Random Forest Model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

rf_mse = mean_squared_error(y_test, y_pred_rf)
rf_r2 = r2_score(y_test, y_pred_rf)
print(f"Random Forest - MSE: {rf_mse:.2f}, R2: {rf_r2:.2f}")

In [ ]:
# LightGBM Model
lgb_model = lgb.LGBMRegressor(random_state=42)
lgb_model.fit(X_train_scaled, y_train)
y_pred_lgb = lgb_model.predict(X_test_scaled)

lgb_mse = mean_squared_error(y_test, y_pred_lgb)
lgb_r2 = r2_score(y_test, y_pred_lgb)
print(f"LightGBM - MSE: {lgb_mse:.2f}, R2: {lgb_r2:.2f}")


In [ ]:
models = ['Random Forest', 'LightGBM', 'XGBoost']
mse_scores = [rf_mse, lgb_mse, xgb_mse]
r2_scores = [rf_r2, lgb_r2, xgb_r2]


plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=r2_scores, palette='viridis')
plt.title('Model Comparison - R2 Scores')
plt.ylabel('R2 Score')
plt.show()

# Visualize the comparison of MSE scores
plt.figure(figsize=(10, 6))
sns.barplot(x=models, y=mse_scores, palette='magma')
plt.title('Model Comparison - MSE Scores')
plt.ylabel('Mean Squared Error')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test.index, y_test, label="True AQI", color='blue')
plt.plot(y_test.index, y_pred_xgb, label="XGBoost Prediction", color='green')
plt.legend()
plt.title('AQI Prediction Comparison using XGBoost')
plt.xlabel('Index')
plt.ylabel('AQI')
plt.show()

In [ ]:
#Ensemble Model using Voting Regressor
ensemble_model = VotingRegressor(estimators=[('rf', rf_model), ('lgb', lgb_model), ('xgb', xgb_model)])
ensemble_model.fit(X_train_scaled, y_train)
y_pred_ensemble = ensemble_model.predict(X_test_scaled)
ensemble_mse = mean_squared_error(y_test, y_pred_ensemble)
ensemble_r2 = r2_score(y_test, y_pred_ensemble)
print(f"Ensemble Model - MSE: {ensemble_mse:.2f}, R2: {ensemble_r2:.2f}")


In [ ]:
## next 7 day prediction
for i in range(7):

    latest_data_scaled = scaler.transform(latest_data[['PM2.5', 'PM10', 'NO', 'NOx', 'NH3', 'SO2', 'CO', 'Ozone', 'Benzene', 'Toluene', 'RH', 'WS', 'WD', 'SR', 'BP', 'AT', 'NO2', 'lag_1', 'lag_2']])


    next_day_prediction = ensemble_model.predict(latest_data_scaled)


    predictions.append(next_day_prediction[0])


    latest_data['lag_2'] = latest_data['lag_1']
    latest_data['lag_1'] = next_day_prediction[0]

print("Predicted AQI for the next 7 days:", predictions)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 8), predictions, marker='o', linestyle='-', color='green')
plt.title('Predicted AQI for the Next 7 Days')
plt.xlabel('Day')
plt.ylabel('Predicted AQI')
plt.xticks(range(1, 8))
plt.grid(True)
plt.show()